<a href="https://colab.research.google.com/github/Morty00933/python/blob/main/2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Установка PyTorch и torchvision
!pip install torch torchvision

# Импорт библиотек
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Задайте параметры обучения
learning_rate = 0.001
epochs = 10
batch_size = 64

# Загрузите данные MNIST
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, transform=transform, download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# Определите архитектуру нейронной сети
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Инициализируйте модель, функцию потерь и оптимизатор
input_size = 28 * 28  # размер изображения MNIST
hidden_size = 128
output_size = 10  # количество классов (цифр от 0 до 9)

model = SimpleNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
for epoch in range(epochs):
    for inputs, labels in train_loader:
        inputs = inputs.view(-1, input_size)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

# Создайте директорию для сохранения обученных весов
save_dir = 'model_weights'
os.makedirs(save_dir, exist_ok=True)

# Сохранение обученных весов модели
torch.save(model.state_dict(), os.path.join(save_dir, 'your_trained_model.pth'))


Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 9912422/9912422 [00:00<00:00, 49723682.47it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 28881/28881 [00:00<00:00, 1866440.08it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 1648877/1648877 [00:00<00:00, 13745352.02it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable



100%|██████████| 4542/4542 [00:00<00:00, 3850925.56it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Epoch [1/10], Loss: 0.5804
Epoch [2/10], Loss: 0.0467
Epoch [3/10], Loss: 0.0312
Epoch [4/10], Loss: 0.1821
Epoch [5/10], Loss: 0.0257
Epoch [6/10], Loss: 0.0209
Epoch [7/10], Loss: 0.0197
Epoch [8/10], Loss: 0.0774
Epoch [9/10], Loss: 0.0020
Epoch [10/10], Loss: 0.0149


In [ ]:
from PIL import Image
import torch
import torch.nn as nn
from torchvision import transforms
import os

# Проверка доступности CUDA
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Определение конфигурируемых параметров
input_size = 28 * 28  # размер изображения MNIST
hidden_size = 128
output_size = 10  # количество классов (цифр от 0 до 9)

# Определение архитектуры нейронной сети
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Инициализация модели
model = SimpleNN(input_size, hidden_size, output_size).to(device)

# Проверка наличия файла с весами модели перед загрузкой
model_weights_path = os.path.join('model_weights', 'your_trained_model.pth')
if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path, map_location=device))
    model.eval()
else:
    print(f"Error: Model weights file '{model_weights_path}' not found.")
    exit()

# Подготовка собственного изображения для тестирования
def preprocess_image(image_path):
    try:
        image = Image.open(image_path).convert('L')  # Преобразование в оттенки серого
    except Exception as e:
        print(f"Error: Unable to open image at '{image_path}'. {str(e)}")
        exit()

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    image = transform(image).view(1, -1)  # Преобразование изображения в тензор
    return image

# Передача изображения через модель для получения прогноза
image_path = '1.png'  # Замените на путь к вашему изображению
if os.path.exists(image_path):
    input_image = preprocess_image(image_path).to(device)

    with torch.no_grad():
        output = model(input_image)
        _, predicted_class = torch.max(output.data, 1)

    print(f'Predicted Class: {predicted_class.item()}')
else:
    print(f"Error: Image file '{image_path}' not found.")


Predicted Class: 8
